In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/nlp-2021-vda/evaluate_models/
!pwd

In [ ]:
!pip install -U nltk

If you want to run evaluation on our sampled responses, download data from google cloud storage.

In [ ]:
import os
if not os.path.isdir('./sampled_responses'):
    !gsutil -m cp -r gs://nlp-lab/evaluation/sampled_responses ./

In [ ]:
import json
import nltk
from nltk.translate.meteor_score import meteor_score
import matplotlib.pyplot as plt
import numpy as np
nltk.download('wordnet')

Plot METEOR Score vs (different supervised and zero shot models)

In [ ]:
path_list = ['124M_zeroshot_0t', '124M_zeroshot_0.5t', '124M_zeroshot_0.7t', 
             '355M_zeroshot_0t', '355M_zeroshot_0.5t', '355M_zeroshot_0.7t',
             '124M_supervised_0t', '124M_supervised_0.5t', '124M_supervised_0.7t',
             '355M_supervised_0t', '355M_supervised_0.5t', '355M_supervised_0.7t']
file_scores = []
gold_path = 'sampled_responses/supervised-baselines/gold.json'

In [ ]:
for path in path_list:
  with open(gold_path) as f:
    gold = json.load(f)

  with open(f"sampled_responses/supervised-baselines/{path}.json") as f:
    data = json.load(f)
  scores = []
  for i in range(len(gold)):
    score = meteor_score([gold[i]['gold_response']], data[i]['sample0'])
    scores.append(score)
  file_scores.append(sum(scores) / len(scores))

In [ ]:
labels = ['124M_zeroshot', '355M_zeroshot', '124M_supervised', '355M_supervised']
temperature_0 = file_scores[::3]
temperature_5 = file_scores[1::3]
temperature_7 = file_scores[2::3]

barWidth = 0.25

r1 = np.arange(len(temperature_0))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]

plt.figure(figsize=(10,5))
plt.bar(r1, temperature_0, width=barWidth, edgecolor='white', label='Temperature=0')
plt.bar(r2, temperature_5, width=barWidth, edgecolor='white', label='Temperature=0.5')
plt.bar(r3, temperature_7, width=barWidth, edgecolor='white', label='Temperature=0.7')

plt.xticks([r + barWidth for r in range(len(temperature_0))], labels)

plt.ylabel("METEOR Score")
plt.legend()
plt.show()